In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate
from tensorflow.keras.models import Model

# Carga de datos

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Intro IA/Proyecto/train.csv', index_col='ID')
df.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
ID,,,,,,,,,,,,,,,,
904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


# Exploración y tratamiento de datos

In [ ]:
nan = df.isna().sum()
nan[nan!=0]

ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_TIENEINTERNET.1              26629
FAMI_EDUCACIONMADRE               23664
dtype: int64

In [ ]:
for c in df.columns:
    print ("%20s"%c, df[c].dtype)

             PERIODO int64
 ESTU_PRGM_ACADEMICO object
ESTU_PRGM_DEPARTAMENTO object
ESTU_VALORMATRICULAUNIVERSIDAD object
ESTU_HORASSEMANATRABAJA object
FAMI_ESTRATOVIVIENDA object
  FAMI_TIENEINTERNET object
 FAMI_EDUCACIONPADRE object
  FAMI_TIENELAVADORA object
 FAMI_TIENEAUTOMOVIL object
ESTU_PRIVADO_LIBERTAD object
ESTU_PAGOMATRICULAPROPIO object
FAMI_TIENECOMPUTADOR object
FAMI_TIENEINTERNET.1 object
 FAMI_EDUCACIONMADRE object
  RENDIMIENTO_GLOBAL object


In [ ]:
# Se eliminan las filas con nans puesto que son muy pocas
df.dropna(subset=['ESTU_VALORMATRICULAUNIVERSIDAD'], inplace=True)

In [ ]:
print(pd.unique(df['ESTU_VALORMATRICULAUNIVERSIDAD']))

['Entre 5.5 millones y menos de 7 millones'
 'Entre 2.5 millones y menos de 4 millones'
 'Entre 4 millones y menos de 5.5 millones' 'Más de 7 millones'
 'Entre 1 millón y menos de 2.5 millones'
 'Entre 500 mil y menos de 1 millón' 'Menos de 500 mil'
 'No pagó matrícula']


In [ ]:
print(pd.unique(df['FAMI_ESTRATOVIVIENDA']))

['Estrato 3' 'Estrato 4' 'Estrato 5' 'Estrato 2' 'Estrato 1' nan
 'Estrato 6' 'Sin Estrato']


In [ ]:
#rellenando nans de estrato de acuerdo al valor de la matrícula
df.loc[(df['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'No pagó matrícula') & (df['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 2'
df.loc[(df['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Menos de 500 mil') & (df['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 2'
df.loc[(df['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 500 mil y menos de 1 millón') & (df['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 3'
df.loc[(df['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 1 millón y menos de 2.5 millones') & (df['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 4'
df.loc[(df['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 2.5 millones y menos de 4 millones') & (df['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 4'
df.loc[(df['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 4 millones y menos de 5.5 millones') & (df['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 5'
df.loc[(df['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 5.5 millones y menos de 7 millones') & (df['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 6'
df.loc[(df['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Más de 7 millones') & (df['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 6'

# Limpieza

In [ ]:
# Se eilimina la columna de privado de la libertad, muy desbalanceada (alrededor de 500 con 'si', el resto con 'no')
df.drop('ESTU_PRIVADO_LIBERTAD', axis=1, inplace=True)
df.drop('PERIODO', axis=1, inplace=True)
# Se borra columna repetida de si tiene internet
df.drop('FAMI_TIENEINTERNET.1', axis=1, inplace=True)
#Valores que tienen en estas columnas simultanemaente valor nan poseen la mayor parte de las otras columnas en nan, por lo que podrian descartarse
nulos = df['ESTU_VALORMATRICULAUNIVERSIDAD'].isna() & df['ESTU_HORASSEMANATRABAJA'].isna()
result = df[~nulos].copy()
result.loc[:, 'FAMI_TIENEINTERNET'] = result[['FAMI_TIENEINTERNET']].fillna(method='pad')
result.loc[:, 'FAMI_EDUCACIONPADRE'] = result[['FAMI_EDUCACIONPADRE']].fillna(method='pad')
result.loc[:, 'FAMI_TIENELAVADORA'] = result[['FAMI_TIENELAVADORA']].fillna(method='pad')
result.loc[:, 'FAMI_TIENEAUTOMOVIL'] = result[['FAMI_TIENEAUTOMOVIL']].fillna(method='pad')
result.loc[:, 'ESTU_PAGOMATRICULAPROPIO'] = result[['ESTU_PAGOMATRICULAPROPIO']].fillna(method='pad')
result.loc[:, 'FAMI_TIENECOMPUTADOR'] = result[['FAMI_TIENECOMPUTADOR']].fillna(method='pad')
result.loc[:, 'FAMI_EDUCACIONMADRE'] = result[['FAMI_EDUCACIONMADRE']].fillna(method='pad')
result.loc[:, 'ESTU_HORASSEMANATRABAJA'] = result[['ESTU_HORASSEMANATRABAJA']].fillna(method='pad')
result.loc[:, 'ESTU_PRGM_DEPARTAMENTO'] = result[['ESTU_PRGM_DEPARTAMENTO']].fillna(method='pad')

# Programa academico

In [ ]:
# Lidiando con el monton de categorias en carreras
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIERIA.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIERÍA.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIER¿A.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIERÌA.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIIERIA.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMPUTACIÓN.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LICENCIATURA.*', 'LICENCIATURA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*Licenciatura.*', 'LICENCIATURA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ADMINISTRACION.*', 'ADMINISTRACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ADMINISTRACIÓN.*', 'ADMINISTRACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ADMINISTRACI¿N.*', 'ADMINISTRACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ADMINSITRACIÓN.*', 'ADMINISTRACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACIÓN SOCIAL.*', 'COMUNICACION SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACION.*', 'COMUNICACION SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACI¿N SOCIAL.*', 'COMUNICACION SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACIONES.*', 'COMUNICACION SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CONTADURIA.*', 'CONTADURIA PUBLICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CONTADURÍA.*', 'CONTADURIA PUBLICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FINANZAS.*', 'CONTADURIA PUBLICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SALUD LABORAL.*', 'SEGURIDAD Y SALUD EN EL TRABAJO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SALUD OCUPACIONAL.*', 'SEGURIDAD Y SALUD EN EL TRABAJO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SALUD EN EL TRABAJO.*', 'SEGURIDAD Y SALUD EN EL TRABAJO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ZOOTECNIA.*', 'VETERINARIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*VETERINARIA.*', 'VETERINARIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INTERNACIONAL.*', 'NEGOCIOS INTERNACIONALES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMERCIO EXTERIOR.*', 'NEGOCIOS INTERNACIONALES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*JURISPRUDENCIA.*', 'DERECHO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DERECHO.*', 'DERECHO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO GRAFICO.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO VISUAL.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO GRÁFICO.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACI¿¿N GR¿¿FICA.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACION GRAFICA.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO GRÀFICO.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISENO GRAFICO.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DEPORTIV.*', 'PROFESIONAL EN DEPORTE', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DEPORTE.*', 'PROFESIONAL EN DEPORTE', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACION AUDIOVISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*NAVALES.*', 'CIENCIAS MILITARES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MILITARES.*', 'CIENCIAS MILITARES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*NAUTICAS.*', 'CIENCIAS MILITARES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MERCADEO.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PUBLICIDAD.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MARKETING.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MERCADOLOGIA.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PUBLICITARIA.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CIENCIA POLÍTICA.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CIENCIA POLITICA.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CIENCIAS POLÍTICAS.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CIENCIAS POLITICAS.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ESTUDIOS POLITICOS.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LINGUISTICA.*', 'FILOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FILOLOGIA.*', 'FILOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LENGUAS Y CULTURA.*', 'FILOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*BACTERIOLOGIA.*', 'BACTERIOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LENGUAS MODERNAS.*', 'LENGUAS MODERNAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ARCHIVISTICA.*', 'ARCHIVISTICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MODA.*', 'DISEÑO DE MODAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*TEXTIL.*', 'DISEÑO DE MODAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*VESTUARIO.*', 'DISEÑO DE MODAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ALTA COSTURA.*', 'DISEÑO DE MODAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*TRABAJO SOCIAL.*', 'TRABAJO SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CONSTRUCCION.*', 'CONSTRUCCION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DANZARIO.*', 'DANZA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DANZA.*', 'DANZA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*NUTRICIÓN.*', 'NUTRICION Y DIETETICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FILOSOFÍA.*', 'FILOSOFIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FILOSOFIA.*', 'FILOSOFIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ARTES PLÁSTICAS.*', 'ARTES PLASTICAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CINE.*', 'CINE', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO INDUSTRIAL.*', 'DISEÑO INDUSTRIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CRIMINAL.*', 'CRIMINALISTICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PSICOLOGIA.*', 'PSICOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PSICOLOGÍA.*', 'PSICOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PSICÓLOGA.*', 'PSICOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MÚSICA.*', 'MUSICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MUSICA.*', 'MUSICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DIRECCION DE BANDA.*', 'MUSICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GASTRONOMÍA.*', 'GASTRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GASTRONOMIA.*', 'GASTRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GASTRONOM¿A.*', 'GASTRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GASTRONOMICAS.*', 'GASTRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MICROBIOLOGIA.*', 'MICROBIOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MATEMÁTICA.*', 'MATEMATICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MATEMATICA.*', 'MATEMATICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*AUDIOVISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACION VISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACI¿N VISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACI¿N VISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACIÓN VISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LITERA.*', 'LITERATURA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*BIBLIOTECOLOGIA.*', 'BIBLIOTECOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*BIBLIOTECOLOGÍA.*', 'BIBLIOTECOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*TURISMO.*', 'TURISMO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*HOTELERA.*', 'TURISMO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PERIODISMO.*', 'PERIODISMO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO DE INTERIORES.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO DE ESPACIOS.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO INTERIOR.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO INTERIOR.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO Y GESTIÓN DE ESPACIOS.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*HISTORIA.*', 'HISTORIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FOTOGRAFÍA.*', 'FOTOGRAFIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*AGROINDUSTRIA.*', 'AGROINDUSTRIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INTERACTIVO.*', 'DISEÑO INTERACTIVO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CROSSMEDIA.*', 'DISEÑO INTERACTIVO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INSTRUMENTACION QUIRURGICA.*', 'INSTRUMENTACION QUIRURGICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*AGRONOMIA.*', 'AGRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ARTES PLASTICAS.*', 'ARTES PLASTICAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GESTION CULTURAL.*', 'GESTION CULTURAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GESTIÓN CULTURAL.*', 'GESTION CULTURAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SOCIOCULTURAL.*', 'GESTION CULTURAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACIÓN CORPORATIVA.*', 'COMUNICACIÓN CORPORATIVA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*RADIO Y TELEVISION.*', 'RADIO Y TELEVISION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*RADIO Y TELEVISIÓN.*', 'RADIO Y TELEVISION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DIGITAL.*', 'DISEÑO DIGITAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SISTEMAS.*', 'SISTEMAS DE INFORMACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*TRADUCCION.*', 'LENGUAS EXTRANJERAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LENGUAS EXTRANJERAS.*', 'LENGUAS EXTRANJERAS', regex=True)

result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ENFERMERÍA', "ENFERMERIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ECONOMÍA', "ECONOMIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ECONOM¿A', "ECONOMIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('INTRUMENTACION QUIRURGICA', "INSTRUMENTACION QUIRURGICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('CIENCIAS DE LA INFORMACION Y LA DOCUMENTACION', "SISTEMAS DE INFORMACION Y DOCUMENTACION")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('CIENCIAS BIBLICAS', "TEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('TEOLOGÍA', "TEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('PLANEACION Y DESARROLLO SOCIAL', "DESARROLLO TERRITORIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GESTION Y DESARROLLO URBANOS', "DESARROLLO TERRITORIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('URBANISMO', "DESARROLLO TERRITORIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GEOGRAFIA APLICADA A LA ORGANIZACION DEL ESPACIO Y PLANIFICACION REGIONAL', "DESARROLLO TERRITORIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BIOLOGÍA', "BIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BIOLOGIA APLICADA', "BIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BIOLOGIA MARINA', "BIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BIOLOGIA AMBIENTAL', "BIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('OPTOMETRÍA', "OPTOMETRIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ARTES ESCÉNICAS', "ARTE DRAMATICO")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ARTES DE LA ESCENA', "ARTE DRAMATICO")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ARTES ESCENICAS', "ARTE DRAMATICO")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GEOLOG¿A', "GEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GEOLOGÍA', "GEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GEOCIENCIAS', "GEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ODONTOLOGÍA', "ODONTOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ESTADÍSTICA', "ESTADISTICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUIMICA', "QUIMICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUIMICA AMBIENTAL', "QUIMICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUÍMICA', "QUIMICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('FÍSICA', "FISICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUÍMICA INDUSTRIAL', "QUIMICA INDUSTRIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QU¿MICA FARMAC¿UTICA', "QUIMICA FARMACEUTICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUÍMICA FARMACÉUTICA', "QUIMICA FARMACEUTICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('CIENCIAS DE LA COMPUTACION', "INGENIERIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ECOLOGÍA', "ECOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ECOLOGIA DE ZONAS COSTERAS', "ECOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('SOCIOLOGÍA', "SOCIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUIMICA Y FARMACIA', "FARMACIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BELLAS ARTES', "ARTE")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ANTROPOLOGÍA', "ANTROPOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('PEDAGOGÍA', "PEDAGOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('REALIZACIÓN EN ANIMACIÓN', "ANIMACIÓN")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('OCEANOGRAFÍA', "OCEANOGRAFIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('OCEANOGRAFIA FISICA', "OCEANOGRAFIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('TERAPIAS PSICOSOCIALES', "PSICOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('COMUNICACIÓN CORPORATIVA', "COMUNICACION SOCIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('TERAPIA OCUPACIONAL', "SEGURIDAD Y SALUD EN EL TRABAJO")
print(result['ESTU_PRGM_ACADEMICO'].unique())

['ENFERMERIA' 'DERECHO' 'MERCADEO Y PUBLICIDAD' 'ADMINISTRACION'
 'PSICOLOGIA' 'VETERINARIA' 'INGENIERIA' 'TURISMO' 'LICENCIATURA'
 'COMUNICACION SOCIAL' 'CIENCIA POLITICA'
 'SEGURIDAD Y SALUD EN EL TRABAJO' 'MUSICA' 'TRABAJO SOCIAL'
 'CONTADURIA PUBLICA' 'ESTADISTICA' 'MEDICINA' 'ODONTOLOGIA'
 'DISEÑO INDUSTRIAL' 'ECONOMIA' 'ARQUITECTURA' 'NEGOCIOS INTERNACIONALES'
 'DISEÑO GRAFICO' 'QUIMICA FARMACEUTICA' 'INSTRUMENTACION QUIRURGICA'
 'FISIOTERAPIA' 'FARMACIA' 'DANZA' 'OPTOMETRIA' 'SOCIOLOGIA'
 'PROFESIONAL EN DEPORTE' 'LITERATURA' 'CINE' 'NUTRICION Y DIETETICA'
 'TERAPIA RESPIRATORIA' 'FILOSOFIA' 'FILOLOGIA' 'SISTEMAS DE INFORMACION'
 'LENGUAS EXTRANJERAS' 'DESARROLLO TERRITORIAL' 'CRIMINALISTICA'
 'BIOLOGIA' 'SISTEMAS DE INFORMACION Y DOCUMENTACION' 'CIENCIAS MILITARES'
 'BACTERIOLOGIA' 'LENGUAS MODERNAS' 'PERIODISMO' 'ANTROPOLOGIA'
 'AGRONOMIA' 'FONOAUDIOLOGIA' 'PROFESIONAL EN AGRONEGOCIOS' 'TEOLOGIA'
 'DISEÑO DE MODAS' 'ARTES PLASTICAS' 'GEOLOGIA' 'QUIMICA' 'HISTORIA'
 'MICROBIOLO

# Codificación

In [ ]:
def numericEncoding(column, d, orderedValues):
  for category in orderedValues:
    d.loc[:, column] = d[column].replace(category, orderedValues.index(category))

  return d

#Codificación de columnas con one-hot y con labeling

In [ ]:
#Se separa la columna objetivo
goal = result['RENDIMIENTO_GLOBAL']
goal_df = goal.to_frame()
definitive = result.drop(columns=['RENDIMIENTO_GLOBAL'])
definitive.head()

,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE
ID,,,,,,,,,,,,
904256,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,No,Si,Postgrado
645256,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,No,Si,Técnica o tecnológica incompleta
308367,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,No,No,Secundaria (Bachillerato) completa
470353,ADMINISTRACION,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,No,Si,Secundaria (Bachillerato) completa
989032,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,No,Si,Primaria completa


In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False, dtype='float32')
definitive = numericEncoding('FAMI_ESTRATOVIVIENDA', definitive, ['Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6'])
definitive = numericEncoding('FAMI_TIENEINTERNET', definitive, ['No', 'Si'])
definitive = numericEncoding('FAMI_EDUCACIONPADRE', definitive, ['No Aplica', 'No sabe', 'Ninguno', 'Primaria incompleta', 'Primaria completa', 'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa',
                                                    'Técnica o tecnológica incompleta', 'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'])
definitive = numericEncoding('FAMI_TIENELAVADORA', definitive, ['No', 'Si'])
definitive = numericEncoding('FAMI_TIENEAUTOMOVIL', definitive, ['No', 'Si'])
definitive = numericEncoding('FAMI_TIENECOMPUTADOR', definitive, ['No', 'Si'])
definitive = numericEncoding('FAMI_EDUCACIONMADRE', definitive, ['No Aplica', 'No sabe', 'Ninguno', 'Primaria incompleta', 'Primaria completa', 'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa',
                                                    'Técnica o tecnológica incompleta', 'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'])
definitive = numericEncoding('ESTU_HORASSEMANATRABAJA', definitive, ['0', 'Menos de 10 horas', 'Entre 11 y 20 horas', 'Entre 21 y 30 horas', 'Más de 30 horas'])
definitive = numericEncoding('ESTU_VALORMATRICULAUNIVERSIDAD', definitive, ['No pagó matrícula', 'Menos de 500 mil', 'Entre 500 mil y menos de 1 millón', 'Entre 1 millón y menos de 2.5 millones',
                                                                     'Entre 2.5 millones y menos de 4 millones', 'Entre 4 millones y menos de 5.5 millones', 'Entre 5.5 millones y menos de 7 millones',
                                                                     'Más de 7 millones'])


In [ ]:
X_encoded_array = ohe.fit_transform(definitive[['ESTU_PRGM_ACADEMICO', 'ESTU_PAGOMATRICULAPROPIO', 'ESTU_PRGM_DEPARTAMENTO']])
X_encoded = pd.DataFrame(X_encoded_array, columns=ohe.get_feature_names_out(['ESTU_PRGM_ACADEMICO', 'ESTU_PAGOMATRICULAPROPIO', 'ESTU_PRGM_DEPARTAMENTO']))

# Resetear el índice de X_encoded para que coincida con el índice de definitive
X_encoded.reset_index(drop=True, inplace=True)
definitive.reset_index(drop=True, inplace=True)

# Combinar ambos DataFrames
combined_df = pd.concat([definitive, X_encoded], axis=1)
combined_df.drop('ESTU_PRGM_ACADEMICO', axis=1, inplace=True)
combined_df.drop('ESTU_PAGOMATRICULAPROPIO', axis=1, inplace=True)
combined_df.drop('ESTU_PRGM_DEPARTAMENTO', axis=1, inplace=True)
#codificación de la variable objetivo
goal_df = numericEncoding('RENDIMIENTO_GLOBAL', goal_df, ['bajo', 'medio-bajo', 'medio-alto', 'alto'])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
#Convertir a float las columnas
for c in goal_df.columns:
    goal_df[c] = goal_df[c].astype('float32')

for c in combined_df.columns:
    combined_df[c] = combined_df[c].astype('float32')

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def trainRandomForest(X, y):
  rf = RandomForestClassifier(n_estimators=100, max_depth=20, bootstrap=True)
  rf.fit(X, y)
  return rf

In [ ]:
rf = trainRandomForest(Xtr, ytr)
y_pred = rf.predict(Xts)
accuracy_score(yts, y_pred)

0.4109072008704776

In [ ]:
rf = trainRandomForest(Xtr, ytr)
y_pred = logistic_predict(rf, Xts)
accuracy_score(yts, y_pred)

0.40963937356701513

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Intro IA/Proyecto/test.csv', index_col='ID')
test.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE
ID,,,,,,,,,,,,,,,
550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa
98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa
499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta
785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa


In [ ]:
nan = result.isna().sum()
nan[nan!=0]

Series([], dtype: int64)

In [ ]:
test.loc[:, 'ESTU_VALORMATRICULAUNIVERSIDAD'] = test[['ESTU_VALORMATRICULAUNIVERSIDAD']].fillna(method='pad')
test.loc[(test['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'No pagó matrícula') & (test['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 2'
test.loc[(test['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Menos de 500 mil') & (test['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 2'
test.loc[(test['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 500 mil y menos de 1 millón') & (df['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 3'
test.loc[(test['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 1 millón y menos de 2.5 millones') & (test['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 4'
test.loc[(test['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 2.5 millones y menos de 4 millones') & (test['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 4'
test.loc[(test['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 4 millones y menos de 5.5 millones') & (test['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 5'
test.loc[(test['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Entre 5.5 millones y menos de 7 millones') & (test['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 6'
test.loc[(test['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Más de 7 millones') & (test['FAMI_ESTRATOVIVIENDA'].isna()), 'FAMI_ESTRATOVIVIENDA'] = 'Estrato 6'
# Se eilimina la columna de privado de la libertad, muy desbalanceada (alrededor de 500 con 'si', el resto con 'no')
test.drop('ESTU_PRIVADO_LIBERTAD', axis=1, inplace=True)
test.drop('PERIODO', axis=1, inplace=True)
# Se borra columna repetida de si tiene internet
test.drop('FAMI_TIENEINTERNET.1', axis=1, inplace=True)
#Valores que tienen en estas columnas simultanemaente valor nan poseen la mayor parte de las otras columnas en nan, por lo que podrian descartarse
nulos = test['ESTU_VALORMATRICULAUNIVERSIDAD'].isna() & test['ESTU_HORASSEMANATRABAJA'].isna()
result = test[~nulos].copy()
#result.drop('ESTU_HORASSEMANATRABAJA', axis=1, inplace=True)
result.loc[:, 'FAMI_TIENEINTERNET'] = result[['FAMI_TIENEINTERNET']].fillna(method='pad')
result.loc[:, 'FAMI_ESTRATOVIVIENDA'] = result[['FAMI_ESTRATOVIVIENDA']].fillna(method='pad')
result.loc[:, 'FAMI_EDUCACIONPADRE'] = result[['FAMI_EDUCACIONPADRE']].fillna(method='pad')
result.loc[:, 'FAMI_TIENELAVADORA'] = result[['FAMI_TIENELAVADORA']].fillna(method='pad')
result.loc[:, 'FAMI_TIENEAUTOMOVIL'] = result[['FAMI_TIENEAUTOMOVIL']].fillna(method='pad')
result.loc[:, 'ESTU_PAGOMATRICULAPROPIO'] = result[['ESTU_PAGOMATRICULAPROPIO']].fillna(method='pad')
result.loc[:, 'FAMI_TIENECOMPUTADOR'] = result[['FAMI_TIENECOMPUTADOR']].fillna(method='pad')
result.loc[:, 'FAMI_EDUCACIONMADRE'] = result[['FAMI_EDUCACIONMADRE']].fillna(method='pad')
result.loc[:, 'ESTU_HORASSEMANATRABAJA'] = result[['ESTU_HORASSEMANATRABAJA']].fillna(method='pad')
result.loc[:, 'ESTU_PRGM_DEPARTAMENTO'] = result[['ESTU_PRGM_DEPARTAMENTO']].fillna(method='pad')

In [ ]:
# Lidiando con el monton de categorias en carreras
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIERIA.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIERÍA.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIER¿A.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIERÌA.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INGENIIERIA.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMPUTACIÓN.*', 'INGENIERIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LICENCIATURA.*', 'LICENCIATURA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*Licenciatura.*', 'LICENCIATURA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ADMINISTRACION.*', 'ADMINISTRACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ADMINISTRACIÓN.*', 'ADMINISTRACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ADMINISTRACI¿N.*', 'ADMINISTRACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ADMINSITRACIÓN.*', 'ADMINISTRACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACIÓN SOCIAL.*', 'COMUNICACION SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACION.*', 'COMUNICACION SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACI¿N SOCIAL.*', 'COMUNICACION SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACIONES.*', 'COMUNICACION SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CONTADURIA.*', 'CONTADURIA PUBLICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CONTADURÍA.*', 'CONTADURIA PUBLICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FINANZAS.*', 'CONTADURIA PUBLICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SALUD LABORAL.*', 'SEGURIDAD Y SALUD EN EL TRABAJO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SALUD OCUPACIONAL.*', 'SEGURIDAD Y SALUD EN EL TRABAJO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SALUD EN EL TRABAJO.*', 'SEGURIDAD Y SALUD EN EL TRABAJO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ZOOTECNIA.*', 'VETERINARIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*VETERINARIA.*', 'VETERINARIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INTERNACIONAL.*', 'NEGOCIOS INTERNACIONALES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMERCIO EXTERIOR.*', 'NEGOCIOS INTERNACIONALES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*JURISPRUDENCIA.*', 'DERECHO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DERECHO.*', 'DERECHO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO GRAFICO.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO VISUAL.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO GRÁFICO.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACI¿¿N GR¿¿FICA.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACION GRAFICA.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO GRÀFICO.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISENO GRAFICO.*', 'DISEÑO GRAFICO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DEPORTIV.*', 'PROFESIONAL EN DEPORTE', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DEPORTE.*', 'PROFESIONAL EN DEPORTE', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACION AUDIOVISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*NAVALES.*', 'CIENCIAS MILITARES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MILITARES.*', 'CIENCIAS MILITARES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*NAUTICAS.*', 'CIENCIAS MILITARES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MERCADEO.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PUBLICIDAD.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MARKETING.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MERCADOLOGIA.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PUBLICITARIA.*', 'MERCADEO Y PUBLICIDAD', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CIENCIA POLÍTICA.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CIENCIA POLITICA.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CIENCIAS POLÍTICAS.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CIENCIAS POLITICAS.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ESTUDIOS POLITICOS.*', 'CIENCIA POLITICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LINGUISTICA.*', 'FILOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FILOLOGIA.*', 'FILOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LENGUAS Y CULTURA.*', 'FILOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*BACTERIOLOGIA.*', 'BACTERIOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LENGUAS MODERNAS.*', 'LENGUAS MODERNAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ARCHIVISTICA.*', 'ARCHIVISTICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MODA.*', 'DISEÑO DE MODAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*TEXTIL.*', 'DISEÑO DE MODAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*VESTUARIO.*', 'DISEÑO DE MODAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ALTA COSTURA.*', 'DISEÑO DE MODAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*TRABAJO SOCIAL.*', 'TRABAJO SOCIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CONSTRUCCION.*', 'CONSTRUCCION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DANZARIO.*', 'DANZA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DANZA.*', 'DANZA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*NUTRICIÓN.*', 'NUTRICION Y DIETETICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FILOSOFÍA.*', 'FILOSOFIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FILOSOFIA.*', 'FILOSOFIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ARTES PLÁSTICAS.*', 'ARTES PLASTICAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CINE.*', 'CINE', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO INDUSTRIAL.*', 'DISEÑO INDUSTRIAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CRIMINAL.*', 'CRIMINALISTICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PSICOLOGIA.*', 'PSICOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PSICOLOGÍA.*', 'PSICOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PSICÓLOGA.*', 'PSICOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MÚSICA.*', 'MUSICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MUSICA.*', 'MUSICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DIRECCION DE BANDA.*', 'MUSICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GASTRONOMÍA.*', 'GASTRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GASTRONOMIA.*', 'GASTRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GASTRONOM¿A.*', 'GASTRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GASTRONOMICAS.*', 'GASTRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MICROBIOLOGIA.*', 'MICROBIOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MATEMÁTICA.*', 'MATEMATICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*MATEMATICA.*', 'MATEMATICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*AUDIOVISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACION VISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACI¿N VISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACI¿N VISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACIÓN VISUAL.*', 'COMUNICACION AUDIOVISUAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LITERA.*', 'LITERATURA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*BIBLIOTECOLOGIA.*', 'BIBLIOTECOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*BIBLIOTECOLOGÍA.*', 'BIBLIOTECOLOGIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*TURISMO.*', 'TURISMO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*HOTELERA.*', 'TURISMO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*PERIODISMO.*', 'PERIODISMO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO DE INTERIORES.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO DE ESPACIOS.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO INTERIOR.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO INTERIOR.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DISEÑO Y GESTIÓN DE ESPACIOS.*', 'DISEÑO DE INTERIORES', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*HISTORIA.*', 'HISTORIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*FOTOGRAFÍA.*', 'FOTOGRAFIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*AGROINDUSTRIA.*', 'AGROINDUSTRIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INTERACTIVO.*', 'DISEÑO INTERACTIVO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*CROSSMEDIA.*', 'DISEÑO INTERACTIVO', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*INSTRUMENTACION QUIRURGICA.*', 'INSTRUMENTACION QUIRURGICA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*AGRONOMIA.*', 'AGRONOMIA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*ARTES PLASTICAS.*', 'ARTES PLASTICAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GESTION CULTURAL.*', 'GESTION CULTURAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*GESTIÓN CULTURAL.*', 'GESTION CULTURAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SOCIOCULTURAL.*', 'GESTION CULTURAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*COMUNICACIÓN CORPORATIVA.*', 'COMUNICACIÓN CORPORATIVA', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*RADIO Y TELEVISION.*', 'RADIO Y TELEVISION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*RADIO Y TELEVISIÓN.*', 'RADIO Y TELEVISION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*DIGITAL.*', 'DISEÑO DIGITAL', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*SISTEMAS.*', 'SISTEMAS DE INFORMACION', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*TRADUCCION.*', 'LENGUAS EXTRANJERAS', regex=True)
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace(r'.*LENGUAS EXTRANJERAS.*', 'LENGUAS EXTRANJERAS', regex=True)

result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ENFERMERÍA', "ENFERMERIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ECONOMÍA', "ECONOMIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ECONOM¿A', "ECONOMIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('INTRUMENTACION QUIRURGICA', "INSTRUMENTACION QUIRURGICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('CIENCIAS DE LA INFORMACION Y LA DOCUMENTACION', "SISTEMAS DE INFORMACION Y DOCUMENTACION")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('CIENCIAS BIBLICAS', "TEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('TEOLOGÍA', "TEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('PLANEACION Y DESARROLLO SOCIAL', "DESARROLLO TERRITORIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GESTION Y DESARROLLO URBANOS', "DESARROLLO TERRITORIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('URBANISMO', "DESARROLLO TERRITORIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GEOGRAFIA APLICADA A LA ORGANIZACION DEL ESPACIO Y PLANIFICACION REGIONAL', "DESARROLLO TERRITORIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BIOLOGÍA', "BIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BIOLOGIA APLICADA', "BIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BIOLOGIA MARINA', "BIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BIOLOGIA AMBIENTAL', "BIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('OPTOMETRÍA', "OPTOMETRIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ARTES ESCÉNICAS', "ARTE DRAMATICO")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ARTES DE LA ESCENA', "ARTE DRAMATICO")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ARTES ESCENICAS', "ARTE DRAMATICO")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GEOLOG¿A', "GEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GEOLOGÍA', "GEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('GEOCIENCIAS', "GEOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ODONTOLOGÍA', "ODONTOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ESTADÍSTICA', "ESTADISTICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUIMICA', "QUIMICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUIMICA AMBIENTAL', "QUIMICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUÍMICA', "QUIMICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('FÍSICA', "FISICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUÍMICA INDUSTRIAL', "QUIMICA INDUSTRIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QU¿MICA FARMAC¿UTICA', "QUIMICA FARMACEUTICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUÍMICA FARMACÉUTICA', "QUIMICA FARMACEUTICA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('CIENCIAS DE LA COMPUTACION', "INGENIERIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ECOLOGÍA', "ECOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ECOLOGIA DE ZONAS COSTERAS', "ECOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('SOCIOLOGÍA', "SOCIOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('QUIMICA Y FARMACIA', "FARMACIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('BELLAS ARTES', "ARTE")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('ANTROPOLOGÍA', "ANTROPOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('PEDAGOGÍA', "PEDAGOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('REALIZACIÓN EN ANIMACIÓN', "ANIMACIÓN")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('OCEANOGRAFÍA', "OCEANOGRAFIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('OCEANOGRAFIA FISICA', "OCEANOGRAFIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('TERAPIAS PSICOSOCIALES', "PSICOLOGIA")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('COMUNICACIÓN CORPORATIVA', "COMUNICACION SOCIAL")
result.loc[:, 'ESTU_PRGM_ACADEMICO'] = result['ESTU_PRGM_ACADEMICO'].str.replace('TERAPIA OCUPACIONAL', "SEGURIDAD Y SALUD EN EL TRABAJO")

In [ ]:
ohe = OneHotEncoder(sparse=False, dtype='float32')
result = numericEncoding('FAMI_ESTRATOVIVIENDA', result, ['Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6'])
result = numericEncoding('FAMI_TIENEINTERNET', result, ['No', 'Si'])
result = numericEncoding('FAMI_EDUCACIONPADRE', result, ['No Aplica', 'No sabe', 'Ninguno', 'Primaria incompleta', 'Primaria completa', 'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa',
                                          'Técnica o tecnológica incompleta', 'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'])
result = numericEncoding('FAMI_TIENELAVADORA', result, ['No', 'Si'])
result = numericEncoding('FAMI_TIENEAUTOMOVIL', result, ['No', 'Si'])
result = numericEncoding('FAMI_TIENECOMPUTADOR', result, ['No', 'Si'])
result = numericEncoding('FAMI_EDUCACIONMADRE', result, ['No Aplica', 'No sabe', 'Ninguno', 'Primaria incompleta', 'Primaria completa', 'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa',
                                          'Técnica o tecnológica incompleta', 'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'])
result = numericEncoding('ESTU_HORASSEMANATRABAJA', result, ['0', 'Menos de 10 horas', 'Entre 11 y 20 horas', 'Entre 21 y 30 horas', 'Más de 30 horas'])
result = numericEncoding('ESTU_VALORMATRICULAUNIVERSIDAD', result, ['No pagó matrícula', 'Menos de 500 mil', 'Entre 500 mil y menos de 1 millón', 'Entre 1 millón y menos de 2.5 millones',
                                                                     'Entre 2.5 millones y menos de 4 millones', 'Entre 4 millones y menos de 5.5 millones', 'Entre 5.5 millones y menos de 7 millones',
                                                                     'Más de 7 millones'])

In [ ]:
X_encoded_array = ohe.fit_transform(result[['ESTU_PRGM_ACADEMICO', 'ESTU_PAGOMATRICULAPROPIO', 'ESTU_PRGM_DEPARTAMENTO']])
X_encoded = pd.DataFrame(X_encoded_array, columns=ohe.get_feature_names_out(['ESTU_PRGM_ACADEMICO', 'ESTU_PAGOMATRICULAPROPIO', 'ESTU_PRGM_DEPARTAMENTO']))

# Resetear el índice de X_encoded para que coincida con el índice de definitive
X_encoded.reset_index(drop=True, inplace=True)
result.reset_index(drop=True, inplace=True)

# Combinar ambos DataFrames
combined_df_new = pd.concat([result, X_encoded], axis=1)
combined_df_new.drop('ESTU_PRGM_ACADEMICO', axis=1, inplace=True)
combined_df_new.drop('ESTU_PAGOMATRICULAPROPIO', axis=1, inplace=True)
combined_df_new.drop('ESTU_PRGM_DEPARTAMENTO', axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
for c in goal_df.columns:
    goal_df[c] = goal_df[c].astype('float32')
    print("%20s"%c, goal_df[c].dtype)

  RENDIMIENTO_GLOBAL float32


In [ ]:
for c in combined_df_new.columns:
  if(not(c in combined_df.columns)):
    print(c)

In [ ]:
combined_df_new.drop('ESTU_PRGM_ACADEMICO_ARTE DRAMÁTICO', axis=1, inplace=True)
combined_df_new.drop('ESTU_PRGM_ACADEMICO_COMUNICACIÖN SOCIAL', axis=1, inplace=True)
combined_df_new.drop('ESTU_PRGM_ACADEMICO_LICENC.EN EDUCACION BASICA PRIMARIA ENF.EN CIENCIAS NATURALESEDUC.AMBIENTAL MATEMA.LENGUA CASTELLANA', axis=1, inplace=True)
combined_df_new.drop('ESTU_PRGM_ACADEMICO_QUIMICA FARMACÉUTICA', axis=1, inplace=True)

In [ ]:
XTEST = combined_df_new.values[:, :]

In [ ]:
predict = rf.predict(XTEST)
predict

array([0., 1., 3., ..., 2., 3., 3.], dtype=float32)

In [ ]:
test_nuevo = test.reset_index()
test_nuevo.shape

(296786, 13)

In [ ]:
submission = pd.DataFrame([test_nuevo.ID, pd.Series(predict, name='RENDIMIENTO_GLOBAL')]).T

In [ ]:
mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
submission['RENDIMIENTO_GLOBAL'] = submission['RENDIMIENTO_GLOBAL'].map(mapping)
submission['ID'] = submission['ID'].astype('int64')

In [ ]:
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-bajo
2,499179,alto
3,782980,bajo
4,785185,bajo


In [ ]:
submission.to_csv('predictions.csv', index=False)
!head predictions.csv

ID,RENDIMIENTO_GLOBAL
550236,bajo
98545,medio-bajo
499179,alto
782980,bajo
785185,bajo
58495,bajo
705444,alto
557548,alto
519909,medio-bajo
